In [1]:
from astropy.io import ascii, fits
import pylab as plt
%matplotlib inline
from astropy import wcs


import numpy as np
import xidplus
from xidplus import moc_routines
import pickle

In [2]:
#Folder containing maps
imfolder='/Volumes/pdh_storage/dmu_products/dmu19/dmu19_HELP-SPIRE-maps/data/'

pswfits=imfolder+'Herschel-Stripe-82_SPIRE250_v1.1.fits'#SPIRE 250 map
pmwfits=imfolder+'Herschel-Stripe-82_SPIRE350_v1.1.fits'#SPIRE 350 map
plwfits=imfolder+'Herschel-Stripe-82_SPIRE500_v1.1.fits'#SPIRE 500 map




In [3]:
#-----250-------------
hdulist = fits.open(pswfits,memmap=True)
im250phdu=hdulist[0].header
im250hdu=hdulist[1].header

im250=hdulist[1].data*1.0E3 #convert to mJy
nim250=hdulist[2].data*1.0E3 #convert to mJy
w_250 = wcs.WCS(hdulist[1].header)
pixsize250=3600.0*w_250.wcs.cdelt[1] #pixel size (in arcseconds)
hdulist.close()
#-----350-------------
hdulist = fits.open(pmwfits,memmap=True)
im350phdu=hdulist[0].header
im350hdu=hdulist[1].header

im350=hdulist[1].data*1.0E3 #convert to mJy
nim350=hdulist[2].data*1.0E3 #convert to mJy
w_350 = wcs.WCS(hdulist[1].header)
pixsize350=3600.0*w_350.wcs.cdelt[1] #pixel size (in arcseconds)
hdulist.close()
#-----500-------------
hdulist = fits.open(plwfits,memmap=True)
im500phdu=hdulist[0].header
im500hdu=hdulist[1].header 
im500=hdulist[1].data*1.0E3 #convert to mJy
nim500=hdulist[2].data*1.0E3 #convert to mJy
w_500 = wcs.WCS(hdulist[1].header)
pixsize500=3600.0*w_500.wcs.cdelt[1] #pixel size (in arcseconds)
hdulist.close()

In [4]:
hdulist = fits.open('/Volumes/pdh_storage/dmu_products/dmu22/dmu22_Herschel-Stripe-82/data/Herschel-Stripe-82_SPIRE_all.fits')
fcat=hdulist[1].data
hdulist.close()

In [40]:
from astropy.coordinates import SkyCoord
from astropy import units as u
c = SkyCoord(ra=fcat['RA']*u.degree, dec=fcat['Dec']*u.degree)  
import pymoc
moc=pymoc.util.catalog.catalog_to_moc(c,200,10)

In [6]:
x_pix, y_pix = np.meshgrid(np.arange(0, w_250.pixel_shape[0]), np.arange(0, w_250.pixel_shape[1]))


In [7]:
sx_pix = x_pix.flatten()
sy_pix = y_pix.flatten()

In [47]:
%%time
ra, dec = w_250.wcs_pix2world(sx_pix,sy_pix, 0)

CPU times: user 2min 9s, sys: 2min 22s, total: 4min 32s
Wall time: 8min 22s


In [48]:
%%time
ind=xidplus.moc_routines.check_in_moc(ra, dec, moc)

CPU times: user 1min 1s, sys: 1min 24s, total: 2min 25s
Wall time: 4min 57s


In [53]:
ind.sum()/ind.shape[0]

0.4001249977795278

51 secs down to 158ms, thats 

In [49]:
sx_pix.shape[0]


354654300

640 hours estimated time, if is the size of moc thats the problem? test. If using moc in tiles, not problem,

In [24]:
from astropy.coordinates import SkyCoord
from astropy import units as u
c = SkyCoord(ra=fcat['RA'][0:len(fcat)//2]*u.degree, dec=fcat['Dec'][0:len(fcat)//2]*u.degree)  
import pymoc
moc2=pymoc.util.catalog.catalog_to_moc(c,100,15)

In [25]:
%%timeit
ind=xidplus.moc_routines.check_in_moc(ra, dec, moc2)

20.1 s ± 3.64 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


for order 11 it was 91.9 ms, for order 15 its 20.1 seconds